# VAMOS API Guide: 3 Ways to Launch NSGA-II

This notebook demonstrates the three primary ways to run optimization experiments in VAMOS, ranging from quick one-liners to fully reproducible production setups.

1. **Quick API (`run_nsgaii`)**: Best for rapid prototyping and interactive exploration.
2. **Advanced API (`optimize`)**: Best for full programmatic control and custom pipelines.
3. **Configuration Files (YAML)**: Best for reproducibility, automation, and CI/CD.

## 1. Quick API (`run_nsgaii`)

Use this for quick experiments where standard defaults are sufficient.

In [1]:
from vamos.experiment.quick import run_nsgaii

print("Running Quick API...")
result = run_nsgaii(
    "zdt1", 
    max_evaluations=5000, 
    pop_size=100, 
    seed=1
)

print(f"Done!")
print(result.summary())

Running Quick API...
Done!
=== VAMOS Quick Result ===
Algorithm: NSGAII
Solutions: 99
Objectives: 2
Evaluations: 5000
Seed: 1

Objective ranges:
  f1: [0.000000, 0.983372]
  f2: [0.102352, 1.743807]

Hypervolume (auto ref): 1.583149
None


## 2. Advanced API (`optimize` + `NSGAIIConfig`)

Use this when you need fine-grained control over operators, parameters, or when building complex application logic.

In [2]:
from vamos import optimize, NSGAIIConfig, OptimizeConfig
from vamos.foundation.problems_registry import ZDT1

print("Running Advanced API...")

# 1. Define Problem
problem = ZDT1(n_var=30)

# 2. Configure Algorithm
algo_config = (
    NSGAIIConfig()
    .pop_size(100)
    # Precise operator control
    .crossover("sbx", prob=0.9, eta=15.0)
    .mutation("pm", prob="1/n", eta=20.0)
    .selection("tournament")
    .survival("rank_crowding")
    .engine("numpy")
    .fixed()
)

# 3. Create Optimization Configuration
config = OptimizeConfig(
    problem=problem,
    algorithm="nsgaii",
    algorithm_config=algo_config,
    termination=("n_eval", 5000),
    seed=1
)

# 4. Run Optimization
result = optimize(config)
result.summary()

Running Advanced API...
=== Optimization Result ===
Solutions: 79
Objectives: 2

Objective ranges:
  f1: [0.000000, 0.974452]
  f2: [0.139288, 1.995860]


## 3. Configuration Files (YAML + CLI)

Use this for reproducible scaling, batch processing, and sharing experiment specifications.

In [3]:
import yaml

spec = {
    "defaults": {
        "algorithm": "nsgaii",
        "problem": "zdt1",
        "max_evaluations": 5000,
        "seed": 1,
        "nsgaii": {
            "crossover": {"method": "sbx", "eta": 15.0},
            "mutation": {"method": "pm", "prob": "1/n"}
        }
    }
}

with open("demo_spec.yaml", "w") as f:
    yaml.dump(spec, f)

print("Created demo_spec.yaml")

Created demo_spec.yaml


In [4]:
!python -m vamos.experiment.cli.main --config demo_spec.yaml

VAMOS Experiment Runner
Problem: ZDT1
Description: Classic bi-objective benchmark with a convex Pareto front.
Decision variables: 30
Objectives: 2
Encoding: continuous
Algorithm: NSGAII
Backend: numpy
Population size: 100
Offspring population size: 100
Max evaluations: 5000
--------------------------------------------------------------------------------
nsgaii -> Time: 356.65 ms | Eval/s: 14019.4
Objective 1 spread: 0.974452

Results stored in: results\ZDT1\nsgaii\numpy\seed_1

Hypervolume (reference [1.074 2.195]): 1.832445
Pareto front plot saved to: results\ZDT1\pareto_front_20251227-185934.png


In [5]:
# Cleanup
import os
if os.path.exists("demo_spec.yaml"):
    os.remove("demo_spec.yaml")